# <font color="ffc800"> **[Piper](https://github.com/rhasspy/piper) model exporter.**
## ![Piper logo](https://contribute.rhasspy.org/img/logo.png)
---

* Original notebook by: [rmcpantoja](http://github.com/rmcpantoja)
* Collaborator: [Xx_Nessu_xX](http://github.com/XxNessuxX)
* Fork maintained by: [allyman17](https://github.com/allyman17/piper)

In [ ]:
#@markdown # <font color="ffc800"> **Verify GPU availability.** 🖥️
#@markdown ---
#@markdown Run this cell first to ensure you have GPU access. ONNX export will be significantly slower on CPU.

import subprocess

def check_gpu():
    print("\033[93m" + "="*50)
    print("GPU VERIFICATION")
    print("="*50 + "\033[0m\n")
    
    # Check CUDA availability via nvidia-smi
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free,driver_version', '--format=csv,noheader'],
                                capture_output=True, text=True, timeout=10)
        if result.returncode == 0 and result.stdout.strip():
            gpu_info = result.stdout.strip().split(', ')
            print(f"\033[92m✓ GPU detected!\033[0m")
            print(f"  • Name: {gpu_info[0]}")
            print(f"  • Total memory: {gpu_info[1]}")
            print(f"  • Free memory: {gpu_info[2]}")
            print(f"  • Driver version: {gpu_info[3]}")
            
            # Additional PyTorch CUDA check
            try:
                import torch
                if torch.cuda.is_available():
                    print(f"\n\033[92m✓ PyTorch CUDA support: Available\033[0m")
                    print(f"  • CUDA version: {torch.version.cuda}")
                else:
                    print(f"\n\033[93m⚠ PyTorch installed but CUDA not available yet.\033[0m")
                    print("  This is normal before running the install cell.")
            except ImportError:
                print(f"\n\033[93mℹ PyTorch not yet installed - run the install cell next.\033[0m")
            
            print("\n\033[92m" + "="*50)
            print("Ready to proceed! Run the install cell next.")
            print("="*50 + "\033[0m")
            return True
        else:
            raise Exception("No GPU found")
    except Exception as e:
        print(f"\033[91m✗ No GPU detected!\033[0m")
        print(f"\n\033[93mTo enable GPU in Colab:\033[0m")
        print("  1. Go to Runtime → Change runtime type")
        print("  2. Select 'T4 GPU' under Hardware accelerator")
        print("  3. Click Save and wait for the runtime to restart")
        print("  4. Re-run this cell to verify")
        print("\n\033[91m" + "="*50)
        print("⚠ Export will be VERY slow without GPU!")
        print("="*50 + "\033[0m")
        return False

gpu_available = check_gpu()

In [ ]:
#@markdown # <font color="ffc800"> **Install software.** 📦
#@markdown ---

print("\033[93mInstalling...")
%cd /content
!git clone -q https://github.com/rhasspy/piper
%cd /content/piper/src/python
!pip install -q pip==24.0
!pip install -q cython>=0.29.0 librosa>=0.9.2 numpy>=1.19.0 pytorch-lightning~=1.7.0 torch==1.13.1
!pip install -q onnx onnxruntime-gpu
!bash build_monotonic_align.sh
!pip install -q torchtext==0.14.1
# fixing recent compatibility issues:
!pip install -q torchaudio==0.13.1 torchmetrics==0.11.4 torchvision==0.14.1
!pip install -q --upgrade gdown

print("\033[93mDone!")

In [ ]:
#@markdown # <font color="ffc800"> **Voice package generation section.** 🗣️
#@markdown ---
%cd /content/piper/src/python
import os
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from google.colab import output
guideurl = "https://github.com/rmcpantoja/piper/blob/master/notebooks/wav/en"

#@markdown ---
#@markdown ### *File selection method:*
use_drive_picker = True #@param {type:"boolean"}
#@markdown ↑ **Check to use Google Drive file picker** (easier), or uncheck to paste IDs/URLs manually.

#@markdown ---
#@markdown ### *Manual entry (only if file picker is disabled):*
#@markdown **Drive ID or direct download link of the model:**
model_id = "" #@param {type:"string"}
#@markdown **Drive ID or direct download link of the config.json file:**
config_id = "" #@param {type:"string"}
#@markdown ---

#@markdown ### *Voice package settings:*
#@markdown **Choose the language code (iso639-1 format):**
#@markdown You can see a list of language codes and names [here](https://www.loc.gov/standards/iso639-2/php/English_list.php).

language = "en_US" #@param ["ar_JO", "ca_ES", "cs_CZ", "da_DK", "de_DE", "el_GR", "en_GB", "en_US", "es_ES", "es_LA", "fi_FI", "fr_FR", "grc", "hu_GU", "is_IS", "it_IT", "kk_KZ", "ka_GE", "lb_LU", "nb", "ne", "nl_BE", "no_NO", "pl_PL", "pt_BR", "pt_PT", "ro_RO", "ru_RU", "sk_SK", "sr", "sv_SE", "sw_CD", "tr_TR", "uk_UA", "vi_VN", "zh_CN"]
voice_name = "" #@param {type:"string"}
voice_name = voice_name.lower()
quality = "medium" #@param ["high", "low", "medium", "x-low"]
#@markdown **Do you want to write a model card?** *(Optional.)*
write_model_card = False #@param {type:"boolean"}

#@markdown **Do you want this voice to have a faster response speed?**
streaming = False #@param {type:"boolean"}

# Store selected file paths globally
selected_model_path = None
selected_config_path = None

def start_process(streaming):
    if not os.path.exists("/content/project/model.ckpt"):
        raise Exception("Could not download model! Make sure the file is shareable to everyone.")
    output.eval_js(f'new Audio("{guideurl}/starting.wav?raw=true").play()')
    if not streaming:
        !python -m piper_train.export_onnx "/content/project/model.ckpt" "{export_voice_path}/{export_voice_name}.onnx"
    else:
        !python -m piper_train.export_onnx_streaming "/content/project/model.ckpt" "{export_voice_path}"
    print("\033[93mCompressing...")
    !tar -czvf "{packages_path}/{export_voice_name}.tar.gz" -C "{export_voice_path}" .
    output.eval_js(f'new Audio("{guideurl}/success.wav?raw=true").play()')
    print("\033[93mDone!")

def download_from_id_or_url(file_id_or_url, output_path):
    """Download file from Drive ID or URL."""
    if file_id_or_url.startswith("1"):
        !gdown -q "{file_id_or_url}" -O "{output_path}"
    elif file_id_or_url.startswith("https://drive.google.com/file/d/"):
        !gdown -q "{file_id_or_url}" -O "{output_path}" --fuzzy
    else:
        !wget -q "{file_id_or_url}" -O "{output_path}"

def copy_from_drive(drive_path, output_path):
    """Copy file from mounted Drive path."""
    import shutil
    shutil.copy(drive_path, output_path)

if not streaming:
    export_voice_name = f"{language}-{voice_name}-{quality}"
else:
    export_voice_name = f"{language}-{voice_name}+RT-{quality}"
export_voice_path = "/content/project/voice-"+export_voice_name
packages_path = "/content/project/packages"
if not os.path.exists(export_voice_path):
    os.makedirs(export_voice_path)
if not os.path.exists(packages_path):
    os.makedirs(packages_path)

if use_drive_picker:
    # Mount Google Drive and use file picker
    from google.colab import drive
    drive.mount('/content/drive')
    
    print("\n\033[93m" + "="*50)
    print("FILE PICKER MODE")
    print("="*50 + "\033[0m\n")
    
    # Create file picker widgets
    from google.colab import files
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    
    # File path input widgets
    model_path_widget = widgets.Text(
        value='',
        placeholder='/content/drive/MyDrive/path/to/model.ckpt',
        description='Model:',
        layout=widgets.Layout(width='80%'),
        style={'description_width': 'initial'}
    )
    
    config_path_widget = widgets.Text(
        value='',
        placeholder='/content/drive/MyDrive/path/to/config.json',
        description='Config:',
        layout=widgets.Layout(width='80%'),
        style={'description_width': 'initial'}
    )
    
    # Browse buttons that open file browser
    browse_model_btn = widgets.Button(description='Browse...', button_style='info')
    browse_config_btn = widgets.Button(description='Browse...', button_style='info')
    
    # Output area for file browser
    file_browser_output = widgets.Output()
    
    def create_file_browser(target_widget, file_filter=None):
        """Create an interactive file browser."""
        current_path = ['/content/drive/MyDrive']
        
        with file_browser_output:
            clear_output()
            
            def list_directory(path):
                items = []
                try:
                    for item in sorted(os.listdir(path)):
                        full_path = os.path.join(path, item)
                        if os.path.isdir(full_path):
                            items.append(('📁 ' + item, full_path, True))
                        elif file_filter is None or item.endswith(file_filter):
                            items.append(('📄 ' + item, full_path, False))
                except PermissionError:
                    pass
                return items
            
            def update_browser(path):
                with file_browser_output:
                    clear_output()
                    current_path[0] = path
                    
                    # Header
                    print(f"\033[93mCurrent: {path}\033[0m\n")
                    
                    # Parent directory button
                    if path != '/content/drive/MyDrive':
                        parent_btn = widgets.Button(description='📁 ..', layout=widgets.Layout(width='auto'))
                        parent_btn.on_click(lambda b: update_browser(os.path.dirname(path)))
                        display(parent_btn)
                    
                    # List items
                    items = list_directory(path)
                    for display_name, full_path, is_dir in items:
                        btn = widgets.Button(description=display_name, layout=widgets.Layout(width='auto'))
                        if is_dir:
                            btn.on_click(lambda b, p=full_path: update_browser(p))
                        else:
                            btn.style.button_color = 'lightgreen'
                            def select_file(b, p=full_path):
                                target_widget.value = p
                                with file_browser_output:
                                    clear_output()
                                    print(f"\033[92m✓ Selected: {p}\033[0m")
                            btn.on_click(select_file)
                        display(btn)
                    
                    # Cancel button
                    cancel_btn = widgets.Button(description='Cancel', button_style='danger')
                    cancel_btn.on_click(lambda b: clear_output())
                    display(widgets.HTML('<br>'))
                    display(cancel_btn)
            
            update_browser(current_path[0])
    
    browse_model_btn.on_click(lambda b: create_file_browser(model_path_widget, '.ckpt'))
    browse_config_btn.on_click(lambda b: create_file_browser(config_path_widget, '.json'))
    
    # Start button
    start_btn = widgets.Button(description='Start Export', button_style='success', 
                                layout=widgets.Layout(width='200px', height='40px'))
    status_output = widgets.Output()
    
    def on_start_click(b):
        with status_output:
            clear_output()
            model_path = model_path_widget.value.strip()
            config_path = config_path_widget.value.strip()
            
            if not model_path or not config_path:
                print("\033[91m✗ Please select both model and config files!\033[0m")
                return
            
            if not os.path.exists(model_path):
                print(f"\033[91m✗ Model file not found: {model_path}\033[0m")
                return
            
            if not os.path.exists(config_path):
                print(f"\033[91m✗ Config file not found: {config_path}\033[0m")
                return
            
            print("\033[93mCopying files...\033[0m")
            copy_from_drive(model_path, "/content/project/model.ckpt")
            copy_from_drive(config_path, f"{export_voice_path}/{export_voice_name}.onnx.json")
            
            # Handle streaming config modification
            if streaming:
                with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "r", encoding="utf-8") as f:
                    tmp = f.read()
                new_config = json.loads(tmp)
                new_config["streaming"] = True
                new_config["key"] = export_voice_name
                with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "w", encoding="utf-8") as f_new:
                    json.dump(new_config, f_new, indent=4)
            
            # Handle model card if requested
            if write_model_card:
                with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "r") as file:
                    config = json.load(file)
                sample_rate = config["audio"]["sample_rate"]
                num_speakers = config["num_speakers"]
                
                model_card_text = f'# Model card for {voice_name} ({quality})\n\n* Language: {language}\n* Speakers: {num_speakers}\n* Quality: {quality}\n* Samplerate: {sample_rate}Hz\n\n## Dataset\n\n* URL: \n* License: \n\n## Training\n\nTrained from scratch.'
                with open(f'{export_voice_path}/MODEL_CARD', 'w') as file:
                    file.write(model_card_text)
            
            start_process(streaming)
    
    start_btn.on_click(on_start_click)
    
    # Display the UI
    print("Select your model (.ckpt) and config (.json) files from Google Drive:\n")
    display(widgets.HBox([model_path_widget, browse_model_btn]))
    display(widgets.HBox([config_path_widget, browse_config_btn]))
    display(file_browser_output)
    display(widgets.HTML('<br>'))
    display(start_btn)
    display(status_output)

else:
    # Original manual ID/URL mode
    print("\033[93mDownloading model and config...\033[0m")
    download_from_id_or_url(model_id, "/content/project/model.ckpt")
    download_from_id_or_url(config_id, f"{export_voice_path}/{export_voice_name}.onnx.json")

    if os.path.exists(f"{export_voice_path}/{export_voice_name}.onnx.json") and streaming:
        with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "r", encoding="utf-8") as f:
            tmp = f.read()
        new_config = json.loads(tmp)
        new_config["streaming"] = True
        new_config["key"] = export_voice_name

        with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "w", encoding="utf-8") as f_new:
            json.dump(new_config, f_new, indent=4)

    if write_model_card:
        with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "r") as file:
            config = json.load(file)
        sample_rate = config["audio"]["sample_rate"]
        num_speakers = config["num_speakers"]
        output.eval_js(f'new Audio("{guideurl}/waiting.wav?raw=true").play()')
        text_area = widgets.Textarea(
            description = "Fill in this template and press Start to generate the voice package:",
            value=f'# Model card for {voice_name} ({quality})\n\n* Language: {language} (normalized)\n* Speakers: {num_speakers}\n* Quality: {quality}\n* Samplerate: {sample_rate}Hz\n\n## Dataset\n\n* URL: \n* License: \n\n## Training\n\nTrained from scratch.\nOr finetuned from: ',
            layout=widgets.Layout(width='500px', height='200px')
        )
        button = widgets.Button(description='Start')

        def create_model_card(button):
            model_card_text = text_area.value.strip()
            with open(f'{export_voice_path}/MODEL_CARD', 'w') as file:
                file.write(model_card_text)
            text_area.close()
            button.close()
            output.clear()
            start_process(streaming)

        button.on_click(create_model_card)

        display(text_area, button)
    else:
        start_process(streaming)

In [ ]:
#@markdown # <font color="ffc800"> **Download/export your generated voice package.** 📥
#@markdown ---

#@markdown #### *How do you want to export your model?*
export_mode = "upload it to my Google Drive" #@param ["Download the voice package on my device (may take some time)", "upload it to my Google Drive"]
print("\033[93mExporting package...")
if export_mode == "Download the voice package on my device (may take some time)":
    from google.colab import files
    files.download(f"{packages_path}/{export_voice_name}.tar.gz")
    msg = "Please wait a moment while the package is being downloaded."
else:
    voicepacks_folder = "/content/drive/MyDrive/piper voice packages"
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    if not os.path.exists(voicepacks_folder):
        os.makedirs(voicepacks_folder)
    !cp "{packages_path}/{export_voice_name}.tar.gz" "{voicepacks_folder}"
    msg = f"You can find the generated voice package at: {voicepacks_folder}."
print(f"\033[93mDone! {msg}")

# "*I want to test this model! I don't need anything else anymore?*"

No, this is almost the end! Now you can share your generated package to your friends, upload to a cloud storage and/or test it on:
* [The inference notebook](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_inference_(ONNX).ipynb)
  * Run the cells in order for it to work correctly, as well as all the notebooks. Also, the inference notebook will guide you through the process using the enhanced accessibility feature if you wish. It's easy to use. Test it!
* Or through the NVDA screen reader!
  * Download and install the latest version of the [add-on](https://github.com/mush42/piper-nvda/releases).
  * Once the add-on is installed, go to NVDA menu/piper voice manager...
  * In the installed voices page, tab until you find the `Install from local file` button, press enter and select the generated package in your downloads.
  * Once the package is selected and installed, apply the changes and restart NVDA to update the voice list.
* Enjoy your creation!